# Two-dimensional dynamical systems

In [ ]:
from functools import cache
from itertools import product
import matplotlib.pyplot as plt
import numpy as np
import random
from scipy.integrate import solve_ivp

from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

## Competition or Lotka-Volterra (Mathematical Biology Sheet 1 Question 7)

$b_{ij}$ represent the strength of the influence of species $j$ on species $i$. Large positive $b_{ij}$ indicates a strong, negative influence. Both $b_{12}, b_{21} > 0$ indicates a competition model, whereas $b_{12} > 0$, $b_{21} < 0$ could represent a predator-prey (Lotka-Volterra) model with a logistic intrinsic growth.

In [ ]:
def lotka_volterra(t, y, rho, b12, b21):
    n1, n2 = y
    n1_dot = n1 * (1 - n1 - b12 * n2)
    n2_dot = rho * n2 * (1 - b21 * n1 - n2)
    return [n1_dot, n2_dot]

In [ ]:
@interact(rho=widgets.FloatSlider(min=0, max=3, value=1, continuous_update=False),
          b12=widgets.FloatSlider(min=-2, max=2, value=1.5, continuous_update=False),
          b21=widgets.FloatSlider(min=-2, max=2, value=0.5, continuous_update=False))
def lvdemo(rho, b12, b21):
    x_range = [0, 1.5]
    y_range = [0, 1.5]
    tf = 8
    initial_states = [(x, y)
                      for x in np.linspace(*x_range, 11) 
                      for y in np.linspace(*y_range, 11)]
    sols = [solve_ivp(lotka_volterra,
                      [0, tf],
                      i,
                      t_eval=np.linspace(0, tf, int(tf*50)),
                      args=(rho, b12, b21))
            for i in initial_states]

    fig = plt.figure(figsize=(8, 8))
    arrowlen = 0.05
    for s in sols:
        t, n1, n2 = s.t, s.y[0, :], s.y[1, :]
        plt.scatter(n1, n2,
                    s=np.linspace(1, 5, len(n1)),
                    c=t,
                    cmap='viridis'
        )
    try:
        plt.plot([0, 1], [1/b12, 0], 'gray')
        plt.plot([0, 1/b21], [1, 0], 'gray')
    except ZeroDivisionError:
        pass

    ax = plt.gca()
    ax.set_xlabel('n1')
    ax.set_ylabel('n2')
    ax.set_xlim(x_range)
    ax.set_ylim(y_range)
    plt.show()

## Harvesting model (Mathematical Biology Sheet 1 Question 8)

In [ ]:
def harvesting(t, y, alpha, f, eps):
    u, v = y
    if u >= 0 and v >= 0:
        u_dot = u * (1 - v) - eps * u**2 - f
        v_dot = -alpha * v * (1 - u)
        return [u_dot, v_dot]
    return [0, 0]

In [ ]:
@interact(alpha=widgets.FloatSlider(min=0, max=3, value=1, continuous_update=False),
          f=widgets.FloatSlider(min=0, max=2, value=0.1, step=0.05, continuous_update=False),
          eps=widgets.FloatSlider(min=0, max=0.5, value=0.20, step=0.05, continuous_update=False))
def harvesting_demo(alpha, f, eps):
    u_range = [0, 7]
    v_range = [0, 6]
    tf = 5
    initial_states = [(x, y)
                      for x in np.linspace(*u_range, 13) 
                      for y in np.linspace(*v_range, 13)]
    sols = [solve_ivp(harvesting,
                      [0, tf],
                      i,
                      t_eval=np.linspace(0, tf, int(tf*50)),
                      args=(alpha, f, eps))
            for i in initial_states]

    fig = plt.figure(figsize=(15, 8))
    arrowlen = 0.05
    for s in sols:
        t, u, v = s.t, s.y[0, :], s.y[1, :]
        plt.scatter(u, v,
                    s=np.linspace(1, 20, len(t)),
                    c=t,
                    cmap='viridis'
        )
        
    plt.plot([1, 1], v_range, 'gray')
    us = np.linspace(*u_range)
    plt.plot(us, -eps * us + 1 - f / us, 'gray')

    ax = plt.gca()
    ax.set_xlabel('u')
    ax.set_ylabel('v')
    ax.set_xlim(u_range)
    ax.set_ylim(v_range)
    if 0 < f < eps:
        title = '(a)'
    elif eps < f < 1 - eps:
        title = '(b)'
    elif 1 - eps < f < 1/(4*eps):
        title = '(c)'
    else:
        title = '(d)'
    ax.set_title(title)
    ax.set_aspect(1/2)
    plt.show()

## SIR model

In [ ]:
def sir(t, y, delta):
    susc, inf = y
    return [-susc * inf, susc * inf - delta * inf]

In [ ]:
@interact(delta=widgets.FloatSlider(min=0, max=2, value=1, continuous_update=False))
def sir_demo(delta):
    x_range = [0, 1]
    y_range = [0, 1]
    tf = 10
    initial_states = [(susc, 1 - susc)
                      for susc in np.linspace(*x_range, 21)]
    sols = [solve_ivp(sir, [0, tf], i,
                      t_eval=np.linspace(0, tf, int(tf*50)),
                     args=(delta,))
            for i in initial_states]

    fig = plt.figure(figsize=(8, 8))
    arrowlen = 0.05
    for s in sols:
        t, susc, inf = s.t, s.y[0, :], s.y[1, :]
        plt.scatter(susc, inf,
            s=np.linspace(1, 30, len(t)),
            c=t, cmap='viridis'
        )

    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    ax.set_xlim(x_range)
    ax.set_ylim(y_range)
    # ax.grid(color='black')
    plt.show()

## SIRS model (Mathematical Biology Sheet 2 Question 1)

In [ ]:
def sir(t, y, delta, gamma):
    susc, inf = y
    return [-susc * inf + gamma * (1 - susc - inf), susc * inf - delta * inf]

In [ ]:
@interact(delta=widgets.FloatSlider(min=0, max=2, value=1, continuous_update=False),
          gamma=widgets.FloatSlider(min=0, max=2, value=0, continuous_update=False))
def sir_demo(delta, gamma):
    x_range = [0, 1]
    y_range = [0, 1]
    tf = 30
    initial_states = [(susc, 1 - susc)
                      for susc in np.linspace(*x_range, 21)]
    sols = [solve_ivp(sir, [0, tf], i,
                      t_eval=np.linspace(0, tf, int(tf*50)),
                     args=(delta, gamma))
            for i in initial_states]

    fig = plt.figure(figsize=(8, 8))
    arrowlen = 0.05
    for s in sols:
        t, susc, inf = s.t, s.y[0, :], s.y[1, :]
        plt.scatter(susc, inf,
            s=np.linspace(1, 30, len(t)),
            c=t, cmap='viridis'
        )

    ax = plt.gca()
    ax.set_aspect('equal', adjustable='box')
    ax.set_xlim(x_range)
    ax.set_ylim(y_range)
    # ax.grid(color='black')
    plt.show()

## Phytoplankton and zooplankton (Mathematical Biology Sheet 2 Question 3)

In [ ]:
def plankton(t, y, eps, b, c):
    phyto, zoo = y
    consumption = zoo * phyto**2 / (eps**2 + phyto**2)
    d_phyto = b * phyto * (1 - phyto) - consumption
    d_zoo = c * consumption - zoo
    return d_phyto, d_zoo

In [ ]:
@interact(
    eps=widgets.FloatSlider(min=0, max=1, value=0.25, step=0.05, continuous_update=False),
    b=widgets.FloatSlider(min=0, max=2, value=1.7, continuous_update=False),
    c=widgets.FloatSlider(min=2, max=4, value=2.2, continuous_update=False)
)
def plankton_demo(eps, b, c):
    x_range = [0, 1.6]
    y_range = [0, 2.2]
    xs = np.linspace(*x_range, 17)
    ys = np.linspace(*y_range, 11) 
    
    tf = 10
    
    initial_states = [(x, y)
                      for x in xs
                      for y in np.linspace(0, 0.2, 5)]
    
    sols = [solve_ivp(plankton, [0, tf], i,
                      t_eval=np.linspace(0, tf, int(tf*50)),
                      args=(eps, b, c),
                      method='BDF')
            for i in initial_states]
    
    separatrix = solve_ivp(plankton, [0, tf], (1, 0.01),
                      t_eval=np.linspace(0, tf, int(tf*50)),
                      args=(eps, b, c),
                      method='BDF')

    fig = plt.figure(figsize=(12, 8))
    arrowlen = 0.05
    for s in sols:
        t, susc, inf = s.t, s.y[0, :], s.y[1, :]
        plt.scatter(susc, inf,
            s=np.linspace(1, 12, len(t)),
            c=t, cmap='viridis'
        )
        
    plt.plot(separatrix.y[0, :], separatrix.y[1, :], 'k-')
        
    xs = np.linspace(*x_range, 101)[1:]
    plt.plot(xs, b * (1 - xs) * (eps**2 + xs**2) / xs, 'm--')
    plt.plot([eps / (c - 1)**0.5] * len(ys), ys, 'r-.')
    
    ax = plt.gca()
#     ax.set_aspect('equal', adjustable='box')
    ax.set_xlim(x_range)
    ax.set_ylim(y_range)
    # ax.grid(color='black')
    ax.legend(['separatrix', '$\dot{x} = 0$', '$\dot{y} = 0$'])
    plt.show()